In [117]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import re

print("Tensorflow Version",tf.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Tensorflow Version 2.6.0


In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
!pip install scikit-learn>=0.24
!pip install ktrain # for BERT model

In [120]:
df = pd.read_csv('/content/drive/MyDrive/datagram/data/train_datam.csv')
df

,Unnamed: 0,category_id,pname
0,0,403,"Double concentré de tomates MUTTI, tube de 130g"
1,1,837,Pur jus de pommes pressées Pure Prémium TROPIC...
2,2,485,Rillettes de poulet rôti en cocotte
3,3,363,My Eyes - Taille-crayons 3 diamètres
4,4,591,Brosse à dents Inter Espaces médium AQUAFRESH
...,...,...,...
161875,162161,372,"Lait pour le corps bio à l'aloé véra SO BIO, 4..."
161876,162162,510,"Hacao aux crevettes + 2 sauces soja, 200g"
161877,162163,587,"Studio Line - Indestuctible 9, gel fixation ex..."
161878,162164,411,BISCUITS POIREAU 100G


In [121]:
del df['Unnamed: 0']
df.columns = ['category_id', 'pname']
df.head()

,category_id,pname
0,403,"Double concentré de tomates MUTTI, tube de 130g"
1,837,Pur jus de pommes pressées Pure Prémium TROPIC...
2,485,Rillettes de poulet rôti en cocotte
3,363,My Eyes - Taille-crayons 3 diamètres
4,591,Brosse à dents Inter Espaces médium AQUAFRESH


In [122]:
"""import random
random_idx_list = [random.randint(1,len(df.tweet)) for i in range(10)] # creates random indexes to choose from dataframe
df.loc[random_idx_list,:].head(10) # Returns the rows with the index and display it"""

'import random\nrandom_idx_list = [random.randint(1,len(df.tweet)) for i in range(10)] # creates random indexes to choose from dataframe\ndf.loc[random_idx_list,:].head(10) # Returns the rows with the index and display it'

In [123]:
"""stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"""

'stop_words = stopwords.words(\'english\')\nstemmer = SnowballStemmer(\'english\')\n\ntext_cleaning_re = "@\\S+|https?:\\S+|http?:\\S|[^A-Za-z0-9]+'

In [124]:
"""def preprocess(text, stem=False):
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)"""

'def preprocess(text, stem=False):\n  text = re.sub(text_cleaning_re, \' \', str(text).lower()).strip()\n  tokens = []\n  for token in text.split():\n    if token not in stop_words:\n      if stem:\n        tokens.append(stemmer.stem(token))\n      else:\n        tokens.append(token)\n  return " ".join(tokens)'

In [125]:
"""df.tweet = df.tweet.apply(lambda x: preprocess(x))"""

'df.tweet = df.tweet.apply(lambda x: preprocess(x))'

In [126]:
TRAIN_SIZE = 0.8
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30

In [127]:
df['tweet'] = df['pname'] 
df['label'] = df['category_id']  

In [128]:
train_data, test_data = train_test_split(df, test_size=1-TRAIN_SIZE,
                                         random_state=7) # Splits Dataset into Training and Testing set

test_data = test_data.groupby('category_id').filter(lambda x : (x['category_id'].count()>=10).any())
train_data = train_data.groupby('category_id').filter(lambda x : (x['category_id'].count()>=10).any())                               
print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))

Train Data size: 129504
Test Data size 32205


In [129]:
train_data.head(10)

,category_id,pname,tweet,label
27381,377,Fines mousses au saumon-poulet-dinde-rognons p...,Fines mousses au saumon-poulet-dinde-rognons p...,377
158645,393,"Haricots beurre extra-fins d'Aucy, 2x4/4, 880g","Haricots beurre extra-fins d'Aucy, 2x4/4, 880g",393
44090,349,Savon détachant écologique à l'huile d'olive b...,Savon détachant écologique à l'huile d'olive b...,349
23089,467,*BLEU DES NEIGES 500G,*BLEU DES NEIGES 500G,467
161552,386,Olives vertes farcies au poivron goût piment,Olives vertes farcies au poivron goût piment,386
49802,610,Mouchoirs,Mouchoirs,610
11173,526,Tarte au chocolat Pierron 400g,Tarte au chocolat Pierron 400g,526
87452,363,"Trousse petit modèle, Ca612 DESSANGE","Trousse petit modèle, Ca612 DESSANGE",363
116463,600,Rasoir pour homme sensitive skinguard GILLETTE...,Rasoir pour homme sensitive skinguard GILLETTE...,600
102295,597,"Gel douche tonifiant NATURA SIBERICA, 400ml","Gel douche tonifiant NATURA SIBERICA, 400ml",597


In [130]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.tweet)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 28867


In [131]:
from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.tweet),
                        maxlen = MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.tweet),
                       maxlen = MAX_SEQUENCE_LENGTH)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

Training X Shape: (129504, 30)
Testing X Shape: (32205, 30)


In [132]:
labels = train_data.label.unique().tolist()

In [133]:
encoder = LabelEncoder()
encoder.fit(train_data.label.to_list())

y_train = encoder.transform(train_data.label.to_list())
y_test = encoder.transform(test_data.label.to_list())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (129504, 1)
y_test shape: (32205, 1)


In [134]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-09-25 22:34:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-25 22:34:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-25 22:34:31--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [135]:
GLOVE_EMB = '/content/glove.6B.300d.txt'
EMBEDDING_DIM = 300
LR = 1e-3
BATCH_SIZE = 1024
EPOCHS = 150
MODEL_PATH = '/content/drive/MyDrive/Classroom/best_model.hdf5'

In [136]:
embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
  values = line.split()
  word = value = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [137]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [138]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                          EMBEDDING_DIM,
                                          weights=[embedding_matrix],
                                          input_length=MAX_SEQUENCE_LENGTH,
                                          trainable=False)

In [139]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

In [140]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(embedding_sequences)
x = Conv1D(64, 5, activation='relu')(x)
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(len(labels), activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, outputs)

In [141]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(optimizer=Adam(learning_rate=LR), loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])
ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1,
                                     min_lr = 0.01,
                                     monitor = 'val_loss',
                                     verbose = 1)

In [142]:
print("Training on GPU...") if tf.test.is_gpu_available() else print("Training on CPU...")

Training on GPU...


In [ ]:
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                    validation_data=(x_test, y_test), callbacks=[ReduceLROnPlateau])

Epoch 1/150
127/127 [==============================] - 63s 452ms/step - loss: 4.1543 - accuracy: 0.1715 - val_loss: 2.5806 - val_accuracy: 0.4147
Epoch 2/150
127/127 [==============================] - 57s 449ms/step - loss: 2.5162 - accuracy: 0.4265 - val_loss: 1.9311 - val_accuracy: 0.5456
Epoch 3/150
127/127 [==============================] - 56s 442ms/step - loss: 2.1026 - accuracy: 0.5075 - val_loss: 1.6801 - val_accuracy: 0.5990
Epoch 4/150
127/127 [==============================] - 56s 443ms/step - loss: 1.9043 - accuracy: 0.5461 - val_loss: 1.5439 - val_accuracy: 0.6253
Epoch 5/150
127/127 [==============================] - 56s 444ms/step - loss: 1.7699 - accuracy: 0.5741 - val_loss: 1.4625 - val_accuracy: 0.6438
Epoch 6/150
127/127 [==============================] - 57s 449ms/step - loss: 1.6726 - accuracy: 0.5923 - val_loss: 1.3976 - val_accuracy: 0.6574
Epoch 7/150
127/127 [==============================] - 58s 453ms/step - loss: 1.5996 - accuracy: 0.6078 - val_loss: 1.3200 -

In [ ]:
s, (at, al) = plt.subplots(2,1)
at.plot(history.history['accuracy'], c= 'b')
at.plot(history.history['val_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['LSTM_train', 'LSTM_val'], loc='upper left')

al.plot(history.history['loss'], c='m')
al.plot(history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')

In [ ]:
"""def decode_sentiment(score):
    return np.argmax(score) 


scores = model.predict(x_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]
y_pred_1d"""

In [ ]:
"""import itertools
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    """This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`."""
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=13)
    plt.yticks(tick_marks, classes, fontsize=13)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=17)
    plt.xlabel('Predicted label', fontsize=17)"""

In [ ]:
"""cnf_matrix = confusion_matrix(test_data.label.to_list(), y_pred_1d)
plt.figure(figsize=(6,6))
plot_confusion_matrix(cnf_matrix, classes=test_data.label.unique(), title="Confusion matrix")
plt.show()"""

In [ ]:
"""print(classification_report(list(test_data.label), y_pred_1d))"""

In [ ]:
"""!pip install scikit-plot"""

In [ ]:
"""import scikitplot as skplt

skplt.metrics.plot_confusion_matrix(
    list(test_data.label), 
    y_pred_1d,
    figsize=(12,12),x_tick_rotation=90)"""

In [ ]:
y_pred_glove = (model.predict(x_test) > 0.5).astype("int")



In [ ]:
print("Accuracy is {} for glove embedding.".format(accuracy_score(test_data.label, y_pred_glove)))

In [ ]:
print(classification_report(y_test, y_pred_glove, zero_division = 1))